In [ ]:
import sys

import tensorflow as tf

sys.path.insert(0, r"../utilities")
sys.path.insert(0, r"../preprocessing")

from preprocessor import *
from utils import loss_accuracy_plot

from TransformerTrainer import TransformerTrainer

encoder_tokenizer_name = "en_tokenizer"
decoder_tokenizer_name = "fr_tokenizer"

tf.get_logger().setLevel('WARNING')

In [ ]:
data_dir = "../data"
en_lines, fr_lines = read_data_files(data_dir, ("small_vocab_en", "small_vocab_fr"))

train_data, test_data, prediction_data, tokenizers = preprocess_data(en_lines, fr_lines, [encoder_tokenizer_name, decoder_tokenizer_name])

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
num_layers = 4 # 6
d_model = 128 # 512
dff = 512  # 2048
num_heads = 8 

In [ ]:
trainer = TransformerTrainer(batch_size=BATCH_SIZE,
                             num_layers=num_layers, 
                             d_model=d_model,
                             dff=dff,
                             num_heads=num_heads,
                             tokenizers=tokenizers,
                             predict_every=5)

In [ ]:
losses, accuracy= trainer.train(train_data=train_data,
                                test_data=test_data,
                                prediction_data=prediction_data,
                                epochs=10)

In [ ]:
loss_accuracy_plot(losses, accuracy, "Transformer")